In [151]:
import json
import bz2
import sys
import os
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import re

In [439]:
#specification of dataset reading parameters and folders path 
QUOTEBANK_POLITICIANS_USA=  'quotebank_politicians_USA.json.bz2' # US politician quotes
WIKI_DATA_FILTERED_LABELED = '../../filtered_politician_labeled_v2.json.bz2' # filtered wikidata with interpretations
POLITICIANS_MENTIONS_USA= 'politicians_mentions_USA.json.bz2'#
QUOTEBANK_POLITICIANS_USA_MAPPED = 'quotebank_politicians_USA_mapped.json.bz2'
#qids_df=pd.read_json(WIKI_DATA_FILTERED_LABELED, lines=True)

In [409]:
Q_catalogue = pd.read_csv('../../wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')
# this is not enough; need full!!

In [382]:
politician_df=pd.read_json(WIKI_DATA_FILTERED_LABELED, lines=True)
politician_us_df = politician_df[politician_df['nationality']=='Q30'] #extract US politicians
names_us=politician_us_df[['qid','name','aliases']] #extract qid, name, aliases
names_us['aliases_all'] = [a+[b] for a,b in zip(names_us['aliases'],names_us['name'])]
#aliases_list = [item for sublist in names_us['aliases'].tolist() for item in sublist] + names_us['name'].tolist()
names_us_list = names_us['aliases_all'].tolist()
names_us_reset = names_us.reset_index(drop=True)
aliases_list = [item for sublist in names_us_list for item in sublist] #flatten aliases for pattern extraction
aliases_index = [names_us_list.index(sublist) for sublist in names_us_list for item in sublist] #but save original index before flatenning
pattern = '|'.join(aliases_list)
#qid_list =  names_us['qid'].tolist()

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_28300/2068062660.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names_us['aliases_all'] = [a+[b] for a,b in zip(names_us['aliases'],names_us['name'])]


In [437]:
ck1['mentions'] = np.empty((len(ck1), 0)).tolist()
ck1['mentions_qids'] = np.empty((len(ck1), 0)).tolist()
# ck1['mentions_names'] = np.empty((len(ck1), 0)).tolist()
# ck1 = ck1.drop('mentions_names', 1)

# ck1['mentions_names'] = ck1['mentions_qids'].apply(qid_interpret) #this needs to use complete catalogue to work!
for i, e in enumerate(aliases_list):
    ck1.loc[ck1['quotation'].str.contains(e)]['mentions'].apply(lambda x: x.append(i))
    ck1['mentions_qids'] = ck1['mentions'].apply(list_interpret)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_28300/34496270.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  ck1.loc[ck1['quotation'].str.contains(e)]['mentions'].apply(lambda x: x.append(i))


In [ ]:
ck1['mentions'] = np.empty((len(ck1), 0)).tolist()
ck1['mentions_qids'] = np.empty((len(ck1), 0)).tolist()
# ck1['mentions_names'] = np.empty((len(ck1), 0)).tolist() #interpret qids
# ck1['mentions_names'] = ck1['mentions_qids'].apply(qid_interpret) #this needs to use complete catalogue to work!
# ck1 = ck1.drop('mentions_names', 1)
for i, e in enumerate(aliases_list):
    ck1.loc[ck1['quotation'].str.contains(e)]['mentions'].apply(lambda x: x.append(i))
    ck1['mentions_qids'] = ck1['mentions'].apply(list_interpret)

In [ ]:
ck1.head()

In [252]:
#extraction of political name's mention from the quotes
CHUNKSIZE=10000
try:
    os.remove(POLITICIANS_MENTIONS_USA)
except:
    pass
total_time,total_mentions,chunk_number,nb_quote=0,0,0,0
mentions=[]
with bz2.open(POLITICIANS_MENTIONS_USA, 'wb') as bzout_mention_usa:
    with pd.read_json(QUOTEBANK_POLITICIANS_USA, lines=True, chunksize=CHUNKSIZE, compression='bz2' ) as df_reader:
        for chunk in df_reader:
            t1=time.time()
            quotations_with_mentions = chunk[chunk.quotation.str.contains(pattern)]
            quotations_with_mentions.to_json(path_or_buf=bzout_mention_usa,orient='records',lines=True)
            mentions = len(quotations_with_mentions)
            total_mentions += mentions
            t2=time.time()
            dt=t2-t1
            total_time+=dt
            chunk_number += 1
            print("Dumped {} US political quotations containing mentioning of other US politicians out of {} quotations [quotations/s: {:.2f}]".format(mentions, CHUNKSIZE, CHUNKSIZE / dt))

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_28300/2346674336.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  quotations_with_mentions = chunk[chunk.quotation.str.contains(pattern)]


Dumped 715 US political quotations containing mentioning of other US politicians out of 10000 quotations [quotations/s: 240.07]
Dumped 671 US political quotations containing mentioning of other US politicians out of 10000 quotations [quotations/s: 244.42]
Dumped 697 US political quotations containing mentioning of other US politicians out of 10000 quotations [quotations/s: 234.33]
Dumped 689 US political quotations containing mentioning of other US politicians out of 10000 quotations [quotations/s: 233.30]
Dumped 687 US political quotations containing mentioning of other US politicians out of 10000 quotations [quotations/s: 247.28]
Dumped 676 US political quotations containing mentioning of other US politicians out of 10000 quotations [quotations/s: 244.00]
Dumped 693 US political quotations containing mentioning of other US politicians out of 10000 quotations [quotations/s: 236.59]
Dumped 675 US political quotations containing mentioning of other US politicians out of 10000 quotations

In [284]:
print(f'Found {total_mentions} US political quotations containing mentioning of other US politicians in {total_time:0.1f} s.')

Found 27156 US political quotations containing mentioning of other US politicians in 1649.3 s.


In [406]:
def list_interpret(x):
    """
    Function for list interpretation. e.g. [qid1,qid2] Record undocumented QIDs if not already recorded.
    """
    tmp = []
    for i in x:
        new_qid = names_us_reset['qid'][aliases_index[i]]
        if new_qid not in tmp:
            tmp.append(new_qid)
    return tmp

In [415]:
def qid_interpret(x):
    """
    Function for qid interpretation. Need whole qid catalogue to work!!
    """
    tmp = []
    for i in x:
        try:
            tmp.append(Q_catalogue.loc[i]['Label'])
        except KeyError as e:
            tmp.append(i)
#             if (i not in Unrecorded_Q):
#                 Unrecorded_Q.append(i)
    return tmp

In [443]:
try:
    os.remove(QUOTEBANK_POLITICIANS_USA_MAPPED)
except:
    pass
CHUNKSIZE = 1000
total_time,chunk_number=0,0
with bz2.open(QUOTEBANK_POLITICIANS_USA_MAPPED, 'wb') as bzout_mention_usa:
    with pd.read_json(POLITICIANS_MENTIONS_USA, lines=True, compression='bz2', chunksize = CHUNKSIZE) as df_reader:
        for chunk in df_reader:
            t1=time.time()
            data_output = chunk
            data_output['mentions'] = np.empty((len(data_output), 0)).tolist()
            data_output['mentions_qids'] = np.empty((len(data_output), 0)).tolist()
            # ck1['mentions_names'] = np.empty((len(ck1), 0)).tolist() #interpret qids
            # ck1['mentions_names'] = ck1['mentions_qids'].apply(qid_interpret) #this needs to use complete catalogue to work!
            # ck1 = ck1.drop('mentions_names', 1)
            for i, e in enumerate(aliases_list):
                data_output.loc[data_output['quotation'].str.contains(e)]['mentions'].apply(lambda x: x.append(i))
                data_output['mentions_qids'] = data_output['mentions'].apply(list_interpret)
            data_output.to_json(path_or_buf=bzout_mention_usa,orient='records',lines=True)
            t2=time.time()
            dt=t2-t1
            total_time+=dt
            chunk_number += 1
            print("Mapped {} US political mentionings to the mentioned politicians.  [quotations/s: {:.2f}]".format(chunk_number*CHUNKSIZE, CHUNKSIZE / dt))

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_28300/3352124762.py:18: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data_output.loc[data_output['quotation'].str.contains(e)]['mentions'].apply(lambda x: x.append(i))


Mapped 1000 US political mentionings to the mentioned politicians.  [quotations/s: 4.77]
Mapped 2000 US political mentionings to the mentioned politicians.  [quotations/s: 4.85]
Mapped 3000 US political mentionings to the mentioned politicians.  [quotations/s: 4.82]
Mapped 4000 US political mentionings to the mentioned politicians.  [quotations/s: 4.93]
Mapped 5000 US political mentionings to the mentioned politicians.  [quotations/s: 5.02]
Mapped 6000 US political mentionings to the mentioned politicians.  [quotations/s: 4.84]
Mapped 7000 US political mentionings to the mentioned politicians.  [quotations/s: 4.92]
Mapped 8000 US political mentionings to the mentioned politicians.  [quotations/s: 4.95]
Mapped 9000 US political mentionings to the mentioned politicians.  [quotations/s: 5.00]
Mapped 10000 US political mentionings to the mentioned politicians.  [quotations/s: 4.95]
Mapped 11000 US political mentionings to the mentioned politicians.  [quotations/s: 5.06]
Mapped 12000 US pol

In [ ]:
pd.read_json(POLITICIANS_MENTIONS_USA, lines=True, chunksize=10000, compression='bz2' ) as df_reader:

In [42]:
with pd.read_json(POLITICIANS_MENTIONS_USA, lines=True, chunksize=10000, compression='bz2' ) as df_reader:
    nb=0
    for chunk in df_reader:
        print(chunk)
            

            qid_quote  qid_speaker qid_mentioned
0   2020-02-07-012379     [Q22686]        Q22686
1   2020-01-29-019304   [Q3957020]         Q6279
2   2020-01-29-019304   [Q3957020]       Q311269
3   2020-01-02-019876    [Q185480]        Q22686
4   2020-01-27-036296      [Q6279]         Q6279
5   2020-04-09-027891      [Q1124]         Q1124
6   2020-01-10-048465      [Q6279]         Q6279
7   2020-01-23-056683    [Q359442]         Q6279
8   2020-01-25-036997  [Q18209402]        Q22686
9   2020-02-03-087410    [Q359442]        Q22686
10  2020-01-16-004972     [Q59315]        Q22686
11  2020-02-07-012791  [Q18978140]      Q7793121
12  2020-02-20-010482     [Q22686]       Q461391
13  2020-02-20-010482     [Q22686]       Q170581
14  2020-01-29-035877      [Q4496]         Q4496
15  2020-03-12-028343   [Q1281204]         Q6279
16  2020-01-29-045741      [Q4496]       Q311269
17  2020-01-12-020460    [Q923242]       Q473239
18  2020-01-12-020460    [Q923242]       Q311269
19  2020-03-03-03131

In [41]:
with pd.read_json(QUOTEBANK_POLITICIANS_USA, lines=True, chunksize=100000, compression='bz2' ) as df_reader:
    
    for chunk in df_reader:
        print(chunk) 
        #for q_quote,qid,quote in chunk:
        #print(q_quote)
    
            




                 quoteID                                          quotation  \
0      2020-01-16-000088  [ Department of Homeland Security ] was livid ...   
1      2020-03-19-000276  [ These ] actions will allow households who ha...   
2      2020-01-26-000499  a few of the candidates who will do better in ...   
3      2020-01-20-000982                        a host of other protections   
4      2020-01-03-001237  A lot of Americans [ were ] upset that this wa...   
...                  ...                                                ...   
99995  2020-03-22-043834  Where were the tests when we needed them? Wher...   
99996  2020-01-25-058535  Whether it's OK for the president to sacrifice...   
99997  2020-04-03-080197  While our hope is to return to classrooms befo...   
99998  2020-02-28-089830  While they were working on impeachment, we wer...   
99999  2020-02-01-061939  while this situation poses a very serious glob...   

                      speaker         qids         

KeyboardInterrupt: 

In [39]:
with pd.read_json(WIKI_DATA_FILTERED_LABELED, lines=True, chunksize=100000, compression='bz2' ) as df_reader:
    for chunk in df_reader:
        print(chunk)




              qid                  name gender nationality  \
0            Q207        George W. Bush   male         Q30   
1            Q946           Donald Tusk   male         Q36   
2           Q1058         Narendra Modi   male        Q668   
3           Q1253           Ban Ki-moon   male        Q884   
4           Q3996      V. P. Kalairajan   male        Q668   
...           ...                   ...    ...         ...   
99995  Q105840175  Frederick H. Magison   male         Q30   
99996  Q105840242  James Goodridge Page   male         Q30   
99997  Q105840296   William M. Robinson   male         Q30   
99998  Q105840297         Silas B. Root   male         Q30   
99999  Q105845441         Andres Taimla   male        Q191   

                                                 aliases  \
0      [George Walker Bush, Bush Jr., Dubya, GWB, Bus...   
1                               [Donald Franciszek Tusk]   
2      [Modi, Narendra Bhai, Narendra Damodardas Modi...   
3              